In [ ]:
import glob

import folium
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
from IPython.display import HTML, display  # type: ignore

from automatic_climb_detection.resample_dataframe import resample_dataframe

figsize = (10, 3)
lw = 4
highlightcolor = "orange"
color = "dimgrey"


def clean(ax):
    ax.set(xticks=[], yticks=[])
    ax.grid(False)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["bottom"].set_visible(False)
    ax.spines["left"].set_visible(False)


def get_fig_ax(figsize=figsize):
    fig, ax = plt.subplots(figsize=figsize, dpi=300)
    clean(ax)
    return fig, ax

## What's a climb?

In [ ]:
fig, ax = get_fig_ax()
altitude = [0, 1, 3, 5, 9, 0, 4, 6, 5, 10, 3, 2, 3, 0, 0]
ax.plot(range(len(altitude)), altitude, lw=lw, color=color)

## <span style="color:orange;">This</span> is a climb.


In [ ]:
fig, ax = get_fig_ax()
ax.plot(range(len(altitude)), altitude, lw=lw, color=color)
ax.plot([0, 1, 2, 3, 4], [0, 1, 3, 5, 9], lw=lw, color=highlightcolor)

## And what about <span style="color:orange;">this</span>?

In [ ]:
fig, ax = get_fig_ax()
ax.plot(range(len(altitude)), altitude, lw=lw, color=color)
ax.plot([11, 12], [2, 3], lw=lw, color=highlightcolor)

## <span style="color:orange;">To split</span> or not to split?


In [ ]:
fig, ax = get_fig_ax()
ax.plot(range(len(altitude)), altitude, lw=lw, color=color)
ax.plot([5, 6, 7], [0, 4, 6], lw=lw, color=highlightcolor)
ax.plot([8, 9], [5, 10], lw=lw, color=highlightcolor)

## To split or <span style="color:orange;">not to split</span>?


In [ ]:
fig, ax = get_fig_ax()
ax.plot(range(len(altitude)), altitude, lw=lw, color=color)
ax.plot([5, 6, 7, 8, 9], [0, 4, 6, 5, 10], lw=lw, color=highlightcolor)

## Real data is <span style="color:orange;">messy</span>.


In [ ]:
fig, ax = get_fig_ax()
interpolant = scipy.interpolate.interp1d(
    range(len(altitude)), altitude, fill_value=True
)
x = np.linspace(0, 14, 141)
noise = 0.2 * np.random.randn(len(x))
ax.plot(x, interpolant(x), lw=lw, color=color)
ax.plot(x, interpolant(x) + noise, lw=lw, color=highlightcolor, alpha=0.7)

## Climb detection

## Climb scoring


## Data & Code


## 

In [ ]:
map_obj = folium.Map(
    location=(48.856, 2.3522),  # Paris
    zoom_start=5,
    width="100%",
    height="100%",
)

csv_files = glob.glob("../data/TourDeFrance2022/*.csv")
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    df = resample_dataframe(
        df, interpolation_column="distance", interpolation_step=1000
    )
    # Convert the latitude and longitude columns to a 'geometry' column
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
    # Set the coordinate reference system (CRS) for the GeoDataFrame
    gdf.crs = "EPSG:4326"
    # Create a list of tuples containing latitude and longitude pairs
    route_coords = list(zip(gdf.latitude, gdf.longitude))
    # Add a polyline with the coordinates of the route
    folium.PolyLine(route_coords, color="blue", weight=5, opacity=0.7).add_to(map_obj)


# Set the output cell width to match the map width
display(HTML("<style>.output {width: 100% !important;}</style>"))
# Display the map in Jupyter notebook
map_obj

## Be creative
